In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:32:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:32:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/23 00:32:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/23 00:32:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.sql.functions import col,current_timestamp,count,countDistinct,sum,aggregate,desc,rank
from pyspark.sql.window import Window

In [3]:
races_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/race/race.csv')\
    .filter('race_year in (2019,2020)')\
    .withColumnRenamed('name', 'race_name')\
    .withColumnRenamed('race_timestamp', 'race_date')\
    .select('circuiit_id','race_id',col('race_year'),col('race_name'),col('race_date'))


In [4]:
circuits_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/circuits/circuits.csv')\
        .select('circuit_id',col('location').alias('circuit_location'))


In [5]:
drivers_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/drivers/drivers.csv')\
    .withColumnRenamed('name','driver_name')\
    .withColumnRenamed('number','driver_number')\
    .withColumnRenamed('nationality','driver_nationality')\
    .select('driver_id','driver_name','driver_number','driver_nationality')


In [6]:
constructors_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/constructors/constructors.csv')\
    .select('constructor_id',col('name').alias('team'))


In [7]:
results_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/result/result.csv')\
    .select('race_id','result_id','driver_id','constructor_id','grid','fastest_lap',col('time').alias('race_time'),'points','position')


In [54]:
race_join = circuits_df.join(races_df, circuits_df.circuit_id == races_df.circuiit_id, 'inner')\
    .join(results_df,races_df.race_id == results_df.race_id,'inner')\
    .join(drivers_df,drivers_df.driver_id == results_df.driver_id, 'inner')\
    .join(constructors_df,constructors_df.constructor_id == results_df.constructor_id,'inner')


In [55]:
df1 = race_join .select('race_year','race_name','race_date','circuit_location','driver_name','driver_number','driver_nationality','team','grid','fastest_lap','race_time','points','position')\
    .withColumn('current_date',current_timestamp())\
    .orderBy(race_join.points.desc())


In [ ]:
df1.coalesce(1).write.option('header',True).format('csv').save('../tbls/presentation/race_results_twoyears')

In [9]:
race_result_agg_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/presentation/race_results_twoyears/race_results_twoyears.csv')\
    .filter(col('race_year') == 2020)


In [27]:
race_result_agg_df.select(count('*')).show()

+--------+
|count(1)|
+--------+
|     340|
+--------+



In [28]:
race_result_agg_df.select(countDistinct('race_name')).show()

+-------------------------+
|count(DISTINCT race_name)|
+-------------------------+
|                       17|
+-------------------------+



In [29]:
df1.where('driver_name="Lewis Hamilton"').select(sum('points'), countDistinct('race_name')).show()

+-----------+-------------------------+
|sum(points)|count(DISTINCT race_name)|
+-----------+-------------------------+
|      760.0|                       28|
+-----------+-------------------------+



In [ ]:
race_result_agg_df.groupBy('circuit_location','driver_name')\
    .agg(sum('points').alias('sum_points'),countDistinct('race_name').alias('cnt_distinct_race_name'))\
    .orderBy('circuit_location').show(50)

In [33]:
rnk_df = df1.groupBy('race_year','driver_name')\
    .agg(sum('points').alias('total_points'),countDistinct('race_name').alias('number_of_races'))\
    .orderBy('race_year')


In [36]:
window_spec = Window.partitionBy('race_year').orderBy(desc('total_points'))

rnk_df.withColumn('rank',rank().over(window_spec)).show(50)

+---------+------------------+------------+---------------+----+
|race_year|       driver_name|total_points|number_of_races|rank|
+---------+------------------+------------+---------------+----+
|     2019|    Lewis Hamilton|       413.0|             21|   1|
|     2019|   Valtteri Bottas|       326.0|             21|   2|
|     2019|    Max Verstappen|       278.0|             21|   3|
|     2019|   Charles Leclerc|       264.0|             21|   4|
|     2019|  Sebastian Vettel|       240.0|             21|   5|
|     2019|      Carlos Sainz|        96.0|             21|   6|
|     2019|      Pierre Gasly|        95.0|             21|   7|
|     2019|   Alexander Albon|        92.0|             21|   8|
|     2019|  Daniel Ricciardo|        54.0|             21|   9|
|     2019|      Sergio Pérez|        52.0|             21|  10|
|     2019|      Lando Norris|        49.0|             21|  11|
|     2019|    Kimi Räikkönen|        43.0|             21|  12|
|     2019|   Nico Hülken